In [ ]:
import os
import glob
import sys
import numpy as np
from tqdm import tqdm
from tifffile import imread, imwrite
from vollseg import StarDist3D, UNET, VollSeg3D
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
from pathlib import Path
from n2v.models import N2V

In [ ]:
image_dir = 'data/tiffiles/'

model_dir = 'data/'

save_dir = image_dir + 'Results/'


NoiseModelName = 'ScipyDenoising'
UNETModelName = 'UNETVolumeSeg'
StarModelName = 'VolumeSeg'

unet_model = UNET(config = None, name = UNETModelName, basedir = model_dir)
star_model = StarDist3D(config = None, name = StarModelName, basedir = model_dir)
noise_model = N2V(config=None, name=NoiseModelName, basedir = model_dir)

In [ ]:
Raw_path = os.path.join(image_dir, '*.tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort
#Minimum size in pixels for the cells to be segmented
min_size = 1
#Adjust the number of tiles depending on how good your GPU is, tiling ensures that your image tiles fit into the runtime
#memory 
n_tiles = (2,2,2)
#If your Unet model is weak we will use the denoising model to obtain the semantic segmentation map, set this to False if this
#is the case else set tit o TRUE if you are using Unet to obtain the semantic segmentation map.
dounet = False
#For 2D images we have the option of segmenting RGB->Greyscale, if so set this to true else let it be False
RGB = False
#Set up the axes keyword depending on the type of image you have, if it is a time lapse movie of XYZ images 
#your axes would be TZYX, if it is a timelapse of 2D images the axes would be TYX, for a directory of XYZ images
#the axes is ZYX and for a directory of XY images the axes is YX
axes = 'ZYX'
for fname in filesRaw:
     
     image = imread(fname)
     Name = os.path.basename(os.path.splitext(fname)[0])
     VollSeg( image, unet_model, star_model, noise_model = noise_model, axes = axes, min_size = min_size,  n_tiles = n_tiles, UseProbability = True, save_dir = save_dir, Name = Name, RGB = RGB, dounet = dounet)
